In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import psutil
import os
from tqdm import tqdm
import cv2
from PIL import Image
from skimage import exposure
from skimage.transform import resize
import scipy
import time
from statistics import mean
import tensorflow as tf
import keras as ke
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [ ]:
Tabela_treino = pd.read_csv('D:\RafaelMalanga\spr-x-ray-age/train_age.csv',sep = ',',engine = 'python')
display(Tabela_treino)

In [ ]:
nlin = np.shape(Tabela_treino)[0]   #numero de linhas da minha tabela do excel
hhh = []
tamy = []

for i in tqdm(range(0,nlin)):
    imageID = Tabela_treino['imageId'].iloc[i]      #pega o valor do imageID de cada linha 
    file_path = ("D://RafaelMalanga//spr-x-ray-age//kaggle//kaggle//train//"  + str(imageID).zfill(6) + ".png")
    img = cv2.imread(file_path) #aplica o numero do imageID pra buscar a imagem no diretório com o mesmo numero
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.merge((img,img,img))
    img = cv2.resize(img, (300,300), interpolation = cv2.INTER_AREA)
    img = np.array(img)/255
    hhh.append(img)
    tamy.append(Tabela_treino['age'].iloc[i])
tamy = np.array(tamy)
    

In [ ]:
X_treino, X_val, Y_treino, Y_val = train_test_split(hhh, tamy, test_size=0.2, random_state=50,shuffle=True)

In [ ]:
tamy = 0 #list(Y_all).clear()
hhh.clear()

print(tamy, hhh)
print(np.shape(X_treino), np.shape(X_val), np.shape(Y_treino), np.shape(Y_val))

In [ ]:
Tabela_teste = pd.read_csv("D:\RafaelMalanga\DATASET spr-x-ray-age\sample_submission_age.csv",sep = ',',engine = 'python')
display(Tabela_teste)

In [ ]:
nlin = np.shape(Tabela_teste)[0] #numero de linhas da minha tabela do excel

X_teste = []
Y_teste = []

for i in tqdm(range(0,nlin)):
    imageID = Tabela_teste['imageId'].iloc[i]      #pega o valor do imageID de cada linha 
    file_path = ("D:\\RafaelMalanga\\DATASET spr-x-ray-age\\kaggle\\kaggle\\test\\"  + str(imageID).zfill(6) + ".png")
    img = cv2.imread(file_path) #aplica o numero do imageID pra buscar a imagem no diretório com o mesmo numero
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.merge((img,img,img))
    img = cv2.resize(img, (300,300), interpolation = cv2.INTER_AREA)
    img = np.array(img)/255
    X_teste.append(img)
    Y_teste.append(Tabela_teste['age'].iloc[i])
Y_teste = np.array(Y_teste)

In [ ]:
from keras.applications import EfficientNetB2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
import math
from tensorflow.keras.callbacks import LearningRateScheduler


start_time = time.time()


# Define o número de núcleos a serem usados pelo TensorFlow
num_cores = 6
# Obtém o ID do processo do script Python
pid = os.getpid()
# Define o conjunto de afinidade do processo para limitar o número de núcleos usados
p = psutil.Process(pid)
p.cpu_affinity(list(range(num_cores)))

with tf.device('/cpu:0'):

    np.random.RandomState(1) # Fixar o pseudo-random generator do numpy
    tf.random.set_seed(1) # Fixar o pseudo-random generator do tensorflow
    # def cosine_annealing(epoch):
    #     epochs = 50 # número de épocas
    #     lr_max = 0.01 # learning rate máximo
    #     lr_min = 0.00001 # learning rate mínimo
    #     return lr_min + (lr_max - lr_min) * 0.5 * (1 + math.cos(math.pi * epoch / epochs))


    # criando a base da EfficientNet pré-treinada
    base_model = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

    # adicionando camadas de classificação no topo da base_model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x) # adição da camada de dropout
    x = Dense(512, activation='relu')(x)
    predictions = Dense(1)(x)

    # definindo o modelo final
    model = Model(inputs=base_model.input, outputs=predictions)

    # congelando as camadas da base_model
    for layer in base_model.layers:
        layer.trainable = False

    opt = ke.optimizers.Adam( learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False)

    # aplicando a técnica de Cosine Annealing na taxa de aprendizagem (lr)
    # lr_scheduler = LearningRateScheduler(cosine_annealing)

    # aplicando o EarlyStopping callback
    # early_stopping = tf.keras.callbacks.EarlyStopping(
    #     monitor='val_loss', 
    #     patience=10, 
    #     restore_best_weights=True
    # )

    model.compile(optimizer= opt, loss='mse',metrics=['mae'])  # Compilando o modelo

    history = model.fit(np.array(X_treino), Y_treino,batch_size = 100, epochs = 50, 
    validation_data = (np.array(X_val), Y_val)) #callbacks=[lr_scheduler, early_stopping])
        
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(history.history["loss"], label="treino")
plt.plot(history.history["val_loss"], label="validacao")
plt.xlabel("epocas")
plt.ylabel("Loss")
plt.ylim( bottom = 0, top = 0.5)
plt.legend(loc='upper right');
plt.subplot(1,2,2)
plt.plot(history.history["accuracy"], label="treino")
plt.plot(history.history["val_accuracy"], label="validacao")
plt.xlabel("epocas")
plt.ylabel("Acuracia")
plt.ylim( bottom = 0.9, top = 1)
plt.legend(loc='lower right');

In [ ]:
#salvar o modelo do melhor fold
model.save('model_tranfer_learning.h5')

#comando para carregar o modelo salvo
# model_salvo = ke.models.load_model("modelFaixa1(female)sem_kfold.h5")

In [ ]:
import pickle

# Salvar todas as variáveis em um arquivo pickle
with open("all_variables.pkl", "wb") as f:
    pickle.dump([model, history], f)

# # Carregar as variáveis a partir do arquivo pickle
# with open("all_variables.pkl", "rb") as f:
#     model, history = pickle.load(f)

In [ ]:
# Define o número de núcleos a serem usados pelo TensorFlow
num_cores = 7
# Obtém o ID do processo do script Python
pid = os.getpid()
# Define o conjunto de afinidade do processo para limitar o número de núcleos usados
p = psutil.Process(pid)
p.cpu_affinity(list(range(num_cores)))

with tf.device('/cpu:0'):

    Ypred_treino=model.predict(np.array(X_treino))
    Ypred_val=model.predict(np.array(X_val))
    Ypred_teste=model.predict(np.array(X_teste))

In [ ]:
from sklearn.metrics import mean_absolute_error
from statistics import mean 

print('MAE treino = ', mean_absolute_error(Ypred_treino, Y_treino))
print('MAE teste = ', mean_absolute_error(Ypred_val, Y_val))

In [ ]:
plt.figure(figsize=(10,7))
idx_t=np.argsort(Y_treino)
plt.plot(Y_treino[idx_t],'.')
plt.plot(Ypred_treino[idx_t],'*',alpha=0.15)
plt.legend(['medido','predito'],prop={'size': 15})
plt.title('dados de treino',size=15)
plt.xticks(fontsize=15);plt.xlabel('amostra',size=15)
plt.yticks(fontsize=15);plt.ylabel('idade (meses)',size=15);
plt.grid()
#plt.savefig('img2.png', bbox_inches='tight');
print("MAE = ", mean_absolute_error(Ypred_treino, Y_treino))

In [ ]:
plt.figure(figsize=(10,7))
idx_t=np.argsort(Y_val)
plt.plot(Y_val[idx_t],'.')
plt.plot(Ypred_val[idx_t],'*',alpha=0.55)
plt.legend(['medido','predito'],prop={'size': 15})
plt.title('Dados de teste',size=15)
plt.xticks(fontsize=15);plt.xlabel('amostra',size=15)
plt.yticks(fontsize=15);plt.ylabel('idade (meses)',size=15);
plt.grid()
#plt.savefig('img3.png', bbox_inches='tight');

print("MAE = ", mean_absolute_error(Ypred_val,Y_val))

In [ ]:
for i in range(0,12):
    Age = Y_val[i]
    PredictedAge = Ypred_val[i]
    error_pred = abs(Age - PredictedAge)
    plt.figure();
    plt.imshow(X_val[i]);plt.title("Age = %.2f" %Age +
                "\n Predicted Age = %.2f" %PredictedAge+
                "\n Error Predict = %.2f" %error_pred )

In [ ]:
Tabela_teste_final = Tabela_teste.copy()
Tabela_teste_final['age'] = Ypred_teste

display(Tabela_teste_final)

In [ ]:
# Tabela_teste_final.to_csv("submissionV3_rafael.csv", index=False)